In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [3]:
# !pip install cmath
!pip install torch
!pip install sklearn
!pip install pandas
!pip install librosa
!pip install numpy
!pip install speechpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=436a5b8b8041dd9dc35db74d1d9c85d315dc8f3be8c33b45d4cc335498d4af5a
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers
!pip install jiwer
!pip install pyctcdecode
from pyctcdecode import build_ctcdecoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 99.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.1/401.1 KB 27.4 MB/s eta 0:00:00


In [ ]:
# from sklearn.model_selection import train_test_split
# import pandas as pd
# data = pd.read_csv("/content/drive/MyDrive/PL/metadata_train.csv")

# train,test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
# train.to_csv("/content/drive/MyDrive/PL/train.csv")
# test.to_csv("/content/drive/MyDrive/PL/dev.csv")

In [5]:
from cmath import acos
import torch
from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
import os
import pandas as pd
# import numpy as np
# from torch.utils.data import DataLoader
# from infer import phonetic_embedding
from help import Atention, wav_norm
from char_embedding import tensor_to_text,text_to_tensor
import librosa
import numpy as np

data = pd.read_csv("train.csv")
# print(data['Wav_path'])
sample = data.shape[0]
# cols = ['Wav_path', 'Lyric_path']

class MDD_Dataset(Dataset):

    def __init__(self):
        acoustic_canonical = data
        self.n_samples = sample
        A = acoustic_canonical['Wav_path']
        C = acoustic_canonical['Lyric_path']
        B = acoustic_canonical['Lyric_path'] #output
        

        self.A_data = A 
        self.C_data = C
        self.y_data = B 

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        p = self.A_data[index]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/vocal_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p)
       
        linguistic = text_to_tensor(self.C_data[index])
        linguistic = torch.tensor(linguistic)
        label = text_to_tensor(self.y_data[index])
        label = torch.tensor(label)
        return phonetic, linguistic, label

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



In [ ]:
# from model import Net
# from cProfile import label
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

"""
"""
f = open("/content/drive/MyDrive/PL/vocal_loss.txt", 'a')
MDD = MDD_Dataset()
# print(data)
net = Acoustic_Phonetic_Linguistic()
net.to('cuda')

dev = pd.read_csv("dev.csv")
# net = torch.load('/content/drive/MyDrive/PL/raw_checkpoint.pth14')
# net = net.to('cpu')
train_loader = DataLoader(dataset=MDD,
                          batch_size=1,
                          shuffle=True,
                          num_workers=0)



# convert to an iterator and look at one random sample


ctc_loss = nn.CTCLoss(blank = 95)
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.AdamW(net.parameters(), lr = 0.00001)
# optimizer = optim.SGD(net.parameters(), 0.01, momentum = 0.9)
# optimizer = torch.load('/home/tuht/train_wav2vec/MDD_Checkpoint/checkpoint_optim.pth')
best_WER = 100
for epoch in range(51):  # loop over the dataset multiple times
    net.train().to('cuda')
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        phonetic, linguistic, labels = data
        # acoustic = torch.tensor(0).to('cuda')
        phonetic = phonetic.to('cuda')
        linguistic = linguistic.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(phonetic, linguistic)
        outputs = outputs.unsqueeze(1)
        input_lengths = outputs.shape
        target_lengths = labels.shape
        target = labels
        input_lengths = [input_lengths[0]]
        target_lengths =[target_lengths[1]]
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        outputs = (F.log_softmax(outputs, dim=2))
        loss = ctc_loss(outputs, labels, input_lengths, target_lengths)
        # print(i)
        # print(loss)
        f.write("(" +str(epoch) + "," + str(i) + ")" + "  loss: " + str(loss) + "\n") 
        loss.backward()
        optimizer.step()
    WER = 0
    net.eval().to('cuda')
    for i in range(len(dev)):
        p = dev['Wav_path'][i]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/vocal_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p)
       
        linguistic = text_to_tensor(dev['Lyric_path'][i])
        linguistic = torch.tensor(linguistic)
        label = text_to_tensor(dev['Lyric_path'][i])
        label = torch.tensor(label)
        phonetic = phonetic.to('cuda')
        linguistic = linguistic.to('cuda')
        phonetic = phonetic.unsqueeze(0)
        linguistic = linguistic.unsqueeze(0)
        outputs = net(phonetic, linguistic)
        outputs = outputs.unsqueeze(0)
        x = F.log_softmax(outputs,dim=2)
        labels = ['ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',]
        x = x.detach().cpu().numpy()
        decoder = build_ctcdecoder(
            labels = labels,
            
        )
        x = x.squeeze(0)
        ground_truth = (clean_corpus(dev['Lyric_path'][i]))
        hypothesis = str(decoder.decode(x))
        error = cer(ground_truth, hypothesis)
        WER = WER + error
    print(epoch)
    print(WER)
    if WER/len(dev)<best_WER:
      print("save " + str(epoch))
      # print(epoch)
      # print(WER)
      best_WER = WER/len(dev)
      print(best_WER)
      torch.save(net, '/content/drive/MyDrive/PL/MDD_Checkpoint/vocal_checkpoint.pth')
        
print('Finished Training')


/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/content/drive/MyDrive/PL/linguistic_encoder.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


0
52.75375736369021
save 0
0.49767695626122843
1
49.288158740054634
save 1
0.4649826296231569
2
47.32307157051754
save 2
0.44644407141997683
3
46.95554340466644
save 3
0.44297682457232496
4
46.498704216270774
save 4
0.43866702090821486
5
46.894330646433254
6
46.385324820148256
save 6
0.4375974039636628
7
46.69395897241932
8
46.676703800720986
9
46.502593925173954
10
46.368142367373665
save 10
0.43743530535258174
11
47.050390867576944
12
47.36449345172786
13
46.412670536207095
14
45.88495086922133
save 14
0.432876894992654
15
46.56878661639938
16
46.69317326952706
17
46.33008340764467
18
46.17582464732134
19
46.55855933244203
20
46.24437826726366
21
46.33345028745771
22
46.38678186287316
23
46.390776637734525
24
46.45819691074204
25
46.49673655647713
26
46.19767973565347
27
45.961767916907384
28
46.22840301893968
29
45.962820726602466
30
46.70265488124777
31
46.44815033241391
32
46.7070296171642
33
46.289915073422804
34
46.53564169583599
35
45.73859097819786
save 35
0.4314961413037534
3

KeyboardInterrupt: ignored

In [ ]:
!stat /content/drive/MyDrive/PL/MDD_Checkpoint/vocal_checkpoint.pth

  File: /content/drive/MyDrive/PL/MDD_Checkpoint/vocal_checkpoint.pth
  Size: 58991357  	Blocks: 115218     IO Block: 65536  regular file
Device: 2ah/42d	Inode: 5745        Links: 1
Access: (0600/-rw-------)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2023-01-23 10:10:00.000000000 +0000
Modify: 2023-01-23 10:09:49.000000000 +0000
Change: 2023-01-23 10:10:00.000000000 +0000
 Birth: -
